In [110]:
import yfinance as yf
import numpy as np
import pandas as pd

In [114]:
etf = pd.read_csv('list_etf.csv')
list_etf = etf['TICKER'].values

In [116]:
len(list_etf)

58

In [165]:
prices = pd.DataFrame()
for i in range(len(list_etf)):
    aux_df = pd.DataFrame()
    aux = yf.Ticker(list_etf[i]).history(period="max")['Close']
    aux_df[list_etf[i]] = aux 
    prices = pd.concat([prices,aux_df],axis=1)

- VEA : No data found, symbol may be delisted
- GSIE : No data found, symbol may be delisted
- BGRN : No data found, symbol may be delisted
- IXN : No data found, symbol may be delisted


In [167]:
prices.shape

(6359, 58)

In [168]:
prices

,URTH,VEA,SPDW,IDEV,GSIE,IVLU,IGOV,GWX,IVV,VTV,...,INDA,EMLC,VWOB,PCY,ILF,EWW,EEMA,DVYE,HYEM,AFK
Date,,,,,,,,,,,,,,,,,,,,,
1996-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.261446,NaN,NaN,NaN,NaN
1996-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.261446,NaN,NaN,NaN,NaN
1996-03-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.450043,NaN,NaN,NaN,NaN
1996-03-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.412324,NaN,NaN,NaN,NaN
1996-03-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,6.487765,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-15,126.410004,NaN,38.299999,69.339996,NaN,26.84,53.279999,39.639999,424.929993,139.850006,...,45.009998,31.879999,79.910004,27.760000,31.680000,48.680000,91.970001,40.220001,23.840000,22.450001
2021-06-16,125.739998,NaN,38.049999,68.879997,NaN,26.65,52.770000,39.369999,422.579987,138.880005,...,44.500000,31.459999,79.260002,27.490000,31.320000,47.689999,90.870003,39.889999,23.780001,22.250000
2021-06-17,125.529999,NaN,37.779999,68.279999,NaN,26.43,52.369999,39.040001,422.450012,137.169998,...,44.360001,31.400000,79.480003,27.629999,31.040001,47.259998,91.919998,39.799999,23.799999,22.059999


In [98]:
# In case we need to convert from close price to adjusted close price, but apparently all of them are adjusted 

def calculate_adjusted_prices(df, column):
    """ 
    :param df: DataFrame with raw prices along with dividend and split_ratio
        values
    :param column: String of which price column should have adjusted prices
        created for it
    :return: DataFrame with the addition of the adjusted price column
    """
    adj_column = 'adj_' + column + '_Calc'

    # Reverse the DataFrame order, sorting by date in descending order
    df.sort_index(ascending=False, inplace=True)

    # Stock Splits with value 0 means a 1:1 relationship
    df['Stock Splits'].replace(0,1,inplace=True)
    
    # Populate Values
    price_col = df[column].values
    split_col = df['Stock Splits'].values
    
    dividend_col = df['Dividends'].values
    adj_price_col = np.zeros(len(df.index))
    adj_price_col[0] = price_col[0]
          
    for i in range(1, len(price_col)):
        adj_price_col[i] = round((adj_price_col[i - 1] + adj_price_col[i - 1] *
                   (((price_col[i] * split_col[i - 1]) -
                     price_col[i - 1] -
                     dividend_col[i - 1]) / price_col[i - 1])), 4)

    df[adj_column] = adj_price_col

    # Change the DataFrame order back to dates ascending
    df.sort_index(ascending=True, inplace=True)

    return df

In [169]:
prices.to_csv('etf_prices.csv')